In [1]:
from typing import List, Optional, Union, Tuple
from abstractions.goap.actions import Prerequisites, Goal
from abstractions.goap.entity import Statement, Attribute
from abstractions.goap.nodes import GameEntity, Node, Position
from abstractions.goap.language_state import GoalState
from abstractions.goap.gridmap import GridMap
from abstractions.goap.interactions import Character, Door, Key, Treasure, Floor, Wall, InanimateEntity, IsPickupable, TestItem, Open, Close, Unlock, Lock, Pickup, Drop, Move
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.shapes import Rectangle, Shadow, Radius, RayCast, BlockedRaycast
from abstractions.goap.agents import Agent

def source_target_position_comparison(source: tuple[int,int], target: tuple[int,int]) -> bool:
    """Check if the source entity's position is the same as the target entity's position."""
    if source and target:
        return source == target
    return False

def treasure_in_neighborhood(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the treasure is in the character's neighborhood."""
    if source.node and target and target.node:
        return source.node in target.node.neighbors()
    return False

def key_in_inventory(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the key is in the character's inventory."""
    if target:
        return target in source.inventory
    return False

def is_treasure(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the target entity is a Treasure."""
    return isinstance(target, Treasure)

def is_golden_key(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Golden Key."""
    return isinstance(target, Key) and target.key_name.value == "Golden Key"

def is_door(entity: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Door."""
    return isinstance(entity, Door)

# Example usage
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([Move, Pickup,  Open, Close, Unlock, Lock])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)
grid_map.generate_entity_type_map()

# Goal 1: Check if the character's position is the same as the treasure's position
reach_treasure_goal = Goal(
    name="Reach the treasure",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(comparisons={
                "source_position": ("position", "position", source_target_position_comparison)
            })
        ]
    )
)

# Goal 2: Check if the treasure is in the character's neighborhood
treasure_in_neighborhood_goal = Goal(
    name="Treasure in neighborhood",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(callables=[treasure_in_neighborhood])
        ]
    )
)

# Goal 3: Check if the key is in the character's inventory
key_in_inventory_goal = Goal(
    name="Key in inventory",
    source_entity_id=character.id,
    target_entity_id=key.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_golden_key])],
        source_target_statements=[
            Statement(callables=[key_in_inventory])
        ]
    )
)

# Goal 4: Check if the door is unlocked
door_unlocked_goal = Goal(
    name="Door unlocked",
    source_entity_id=door.id,
    prerequisites=Prerequisites(
        source_statements=[
            Statement(callables=[is_door]),
            Statement(conditions={"is_locked": False})
        ],
        target_statements=[],
        source_target_statements=[]
    )
)

# goals = [reach_treasure_goal, treasure_in_neighborhood_goal, key_in_inventory_goal, door_unlocked_goal]
goals = [key_in_inventory_goal]
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf"
model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF/Nous-Hermes-2-Mixtral-8x7B-DPO.Q8_0.gguf"
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q8_0.gguf"
# model_path = None
agent = Agent(
    goals=goals,
    character_id=character.id,
    actions=grid_map.get_actions(),
    entity_type_map=grid_map.entity_type_map,
    llm = model_path

)
mdp = True
if mdp:
    shape = grid_map.get_rectangle()
else:
    character_node = GameEntity.get_instance(character.id).node
    shape = grid_map.get_shadow(source=character_node, max_radius=5)



pygame-ce 2.4.1 (SDL 2.28.5, Python 3.12.1)


/Users/tommasofurlanello/.pyenv/versions/3.12.1/envs/abstractions/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from /Users/tommasofurlanello/Documents/Dev/models/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF/Nous-Hermes-2-Mixtral-8x7B-DPO.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = emozilla
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embe

Updating block attributes... for door


....................................................................................................
llama_new_context_with_model: n_ctx      = 30016
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: freq_base  = 1000000.0
llama_new_context_with_model: freq_scale = 1
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M3 Max
ggml_metal_init: picking default device: Apple M3 Max
ggml_metal_init: using embedded metal library
ggml_metal_init: GPU name:   Apple M3 Max
ggml_metal_init: GPU family: MTLGPUFamilyApple9  (1009)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction support   = true
ggml_metal_init: simdgroup matrix mul. support = true
ggml_metal_init: hasUnifiedMemory              = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 103079.22 MB
ggml_backend_metal_buffer_type_alloc_buffer: allocat

In [2]:
agent.derive_allowed_strings(grid_map)

['East Move Floor',
 'North Move Floor',
 'NorthEast Move Floor',
 'NorthWest Move Floor',
 'West Move Floor']

In [3]:
agent.allowed_strings_count

{}

In [4]:
agent.run(grid_map, max_steps=10000, mdp=True)


--- Step 0 ---
Generating generator for ['East Move Floor', 'North Move Floor', 'NorthEast Move Floor', 'NorthWest Move Floor', 'West Move Floor']



llama_print_timings:        load time =    4181.68 ms
llama_print_timings:      sample time =       1.70 ms /     7 runs   (    0.24 ms per token,  4127.36 tokens per second)
llama_print_timings: prompt eval time =   14824.67 ms /  2659 tokens (    5.58 ms per token,   179.36 tokens per second)
llama_print_timings:        eval time =     311.28 ms /     6 runs   (   51.88 ms per token,    19.28 tokens per second)
llama_print_timings:       total time =   15177.85 ms /  2665 tokens



Action State:


Generated Action:
NorthEast Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='f01ee57a-7daf-45a2-b283-fddf302f67bc' position=Position(name='Position', id='26668d99-d669-49f0-b1ea-9bc88e6021f9', value=(6, 0)) entities=[Floor(id=8120d091-fecb-4353-92d3-05902261ebb2, name=Floor_6_0, position=(6, 0))] gridmap_id='6e57a90d-d425-4961-83dc-10c87b010391' blocks_movement=BlocksMovement(name='BlocksMovement', id='3ca8642d-9ab2-45e5-8dce-20c16c227cad', value=False) blocks_light=BlocksLight(name='BlocksLight', id='eaef7607-9215-43a7-bd78-d6c18c15aee1', value=False) hash_resolution='default' starting from action string NorthEast Move Floor

Action Payload:
actions=[ActionInstance(source_id='fff06a2b-84b6-4191-8c23-e3f177b07888', target_id='8120d091-fecb-4353-92d3-05902261ebb2', action=Move(name='Move Step', prerequisites=Prerequisites(source_statements=[Statement(name='Statement', id='cae8d5c9-9dde-41b0-8307-4376a2a7401f'


llama_print_timings:        load time =    4181.68 ms
llama_print_timings:      sample time =       0.41 ms /     5 runs   (    0.08 ms per token, 12195.12 tokens per second)
llama_print_timings: prompt eval time =   14635.81 ms /  2756 tokens (    5.31 ms per token,   188.31 tokens per second)
llama_print_timings:        eval time =     205.20 ms /     4 runs   (   51.30 ms per token,    19.49 tokens per second)
llama_print_timings:       total time =   14853.98 ms /  2760 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (5, 1) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_6_0' (6, 0) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (5, 1) -> (6, 0)]
### Natural Language Summary:
The position: (5, 1) of Character 'Player' changed to (6, 0) as a result of Move.


Generated Action:
East Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='6a32e79d-ea15-4e39-ad3d-4e1aefe70a4c' position=Position(name='Position', id='4217ad45-7120-4104-a3af-f0c9f2965f9b', value=(7, 0)) entities=[Floor(id=382fd8e1-d33c-443a-9fcb-196e564b0421, name=Floor_7_0, position=(7, 0))] gridmap_id='6e57a90d-d425-4961-83dc-10c87b010391' blocks_movement=BlocksMovement(name='BlocksMovement', 


llama_print_timings:        load time =    4181.68 ms
llama_print_timings:      sample time =       1.69 ms /     6 runs   (    0.28 ms per token,  3546.10 tokens per second)
llama_print_timings: prompt eval time =   15318.57 ms /  2772 tokens (    5.53 ms per token,   180.96 tokens per second)
llama_print_timings:        eval time =     263.29 ms /     5 runs   (   52.66 ms per token,    18.99 tokens per second)
llama_print_timings:       total time =   15629.95 ms /  2777 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (6, 0) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_7_0' (7, 0) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (6, 0) -> (7, 0)]
### Natural Language Summary:
The position: (6, 0) of Character 'Player' changed to (7, 0) as a result of Move.


Generated Action:
SouthEast Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='15502ed1-4311-4331-9185-8bd6ee1d4058' position=Position(name='Position', id='242daf8e-733c-4838-99af-cfe74c65a327', value=(8, 1)) entities=[Floor(id=5bca90f3-2a84-4618-b458-18b2c5ae9d4a, name=Floor_8_1, position=(8, 1))] gridmap_id='6e57a90d-d425-4961-83dc-10c87b010391' blocks_movement=BlocksMovement(name='BlocksMoveme


llama_print_timings:        load time =    4181.68 ms
llama_print_timings:      sample time =       0.45 ms /     6 runs   (    0.07 ms per token, 13392.86 tokens per second)
llama_print_timings: prompt eval time =   16282.46 ms /  2821 tokens (    5.77 ms per token,   173.25 tokens per second)
llama_print_timings:        eval time =     260.96 ms /     5 runs   (   52.19 ms per token,    19.16 tokens per second)
llama_print_timings:       total time =   16558.05 ms /  2826 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (7, 0) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_8_1' (8, 1) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (7, 0) -> (8, 1)]
### Natural Language Summary:
The position: (7, 0) of Character 'Player' changed to (8, 1) as a result of Move.


Generated Action:
NorthWest Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='6a32e79d-ea15-4e39-ad3d-4e1aefe70a4c' position=Position(name='Position', id='4217ad45-7120-4104-a3af-f0c9f2965f9b', value=(7, 0)) entities=[Floor(id=382fd8e1-d33c-443a-9fcb-196e564b0421, name=Floor_7_0, position=(7, 0))] gridmap_id='6e57a90d-d425-4961-83dc-10c87b010391' blocks_movement=BlocksMovement(name='BlocksMoveme

In [ ]:
# obs_text, act_text, goal_text = agent.generate_text(shape)

In [ ]:
# agent.generate_action(obs_text, act_text, goal_text)

In [ ]:
# agent = Agent(
#     goals=goals,
#     character_id=character.id,
#     actions=grid_map.get_actions(),
#     entity_type_map=grid_map.entity_type_map
# )

# # Run the agent
# agent.run(grid_map, max_steps=10000, mdp=True)

In [ ]:
# gang = Character.get_instance(character.id)
